In [ ]:
!unzip a01c26dcd27711ea.zip

Archive:  a01c26dcd27711ea.zip
replace Dataset/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
import sklearn
import os
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_set=pd.read_csv('Dataset/train.csv')

In [ ]:
train_set.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [ ]:
train_set.drop(columns=['issue_date','listing_date'], inplace=True)

In [ ]:
train_set.head()

,pet_id,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2.0,Black,0.50,11.06,18,4,0.0,1


In [ ]:
#let's use one hot encode to change the color_type categorical variable
Feature = pd.get_dummies(train_set['color_type'])

In [ ]:
Feature 

,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Black Tiger,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Brown Tiger,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,Gray Tabby,Green,Lilac Point,Liver,Liver Tick,Lynx Point,Orange,Orange Tabby,Pink,Red,Red Merle,Red Tick,Sable,Seal Point,Silver,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
18830,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18831,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18832,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
train_set.columns
train_set=train_set[['pet_id', 'condition','length(m)', 'height(cm)', 'X1','X2', 'breed_category', 'pet_category']]
train_set=pd.concat([train_set,Feature],axis=1)
train_set.head()

,pet_id,condition,length(m),height(cm),X1,X2,breed_category,pet_category,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Black Tiger,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Brown Tiger,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,Gray Tabby,Green,Lilac Point,Liver,Liver Tick,Lynx Point,Orange,Orange Tabby,Pink,Red,Red Merle,Red Tick,Sable,Seal Point,Silver,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,ANSL_69903,2.0,0.80,7.78,13,9,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ANSL_66892,1.0,0.72,14.19,13,9,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,ANSL_69750,NaN,0.15,40.90,15,4,2.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ANSL_71623,1.0,0.62,17.82,0,1,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,ANSL_57969,2.0,0.50,11.06,18,4,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_set.shape

(18834, 64)

In [ ]:
train_set.drop(['pet_id'],axis=1,inplace=True)

In [ ]:
columns=train_set.columns


In [ ]:
X=train_set[columns]
X.head()

,condition,length(m),height(cm),X1,X2,breed_category,pet_category,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Black Tiger,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Brown Tiger,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,Gray Tabby,Green,Lilac Point,Liver,Liver Tick,Lynx Point,Orange,Orange Tabby,Pink,Red,Red Merle,Red Tick,Sable,Seal Point,Silver,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,2.0,0.80,7.78,13,9,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,0.72,14.19,13,9,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,NaN,0.15,40.90,15,4,2.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,0.62,17.82,0,1,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,2.0,0.50,11.06,18,4,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Train set has two colours not present in the Test set, let's check its frequency
cnt = X[(X['Brown Tiger'] == 1) | (X['Black Tiger'] == 1)]
cnt

,condition,length(m),height(cm),X1,X2,breed_category,pet_category,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Black Tiger,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Brown Tiger,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,Gray Tabby,Green,Lilac Point,Liver,Liver Tick,Lynx Point,Orange,Orange Tabby,Pink,Red,Red Merle,Red Tick,Sable,Seal Point,Silver,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
2343,1.0,0.15,19.74,0,1,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8830,2.0,0.37,38.58,17,9,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8901,2.0,0.74,33.05,0,7,1.0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13240,2.0,0.22,32.12,13,9,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16908,1.0,0.10,28.01,0,7,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Since are just 5 rows we can drop them
X=X.drop(X[(X['Black Tiger'] == 1) | (X['Brown Tiger'] == 1)].index)
y1=pd.get_dummies(X['pet_category'])
y2=pd.get_dummies(X['breed_category'])
X.head()

,condition,length(m),height(cm),X1,X2,breed_category,pet_category,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Black Tiger,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Brown Tiger,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,Gray Tabby,Green,Lilac Point,Liver,Liver Tick,Lynx Point,Orange,Orange Tabby,Pink,Red,Red Merle,Red Tick,Sable,Seal Point,Silver,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,2.0,0.80,7.78,13,9,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,0.72,14.19,13,9,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,NaN,0.15,40.90,15,4,2.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,0.62,17.82,0,1,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,2.0,0.50,11.06,18,4,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.impute import KNNImputer
#I noticed the are NaN values, let's replace using the KNN imputer 
imputer = KNNImputer(n_neighbors=3, weights='uniform', metric='nan_euclidean')
X=imputer.fit_transform(X)
X=pd.DataFrame(data=X, columns=columns)
X.drop(['breed_category','pet_category','Black Tiger','Brown Tiger'],axis=1,inplace=True)
X['condition']=X['condition'].astype('int')

In [ ]:
X.head()

,condition,length(m),height(cm),X1,X2,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,Gray Tabby,Green,Lilac Point,Liver,Liver Tick,Lynx Point,Orange,Orange Tabby,Pink,Red,Red Merle,Red Tick,Sable,Seal Point,Silver,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,2,0.80,7.78,13.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.72,14.19,13.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,0.15,40.90,15.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.62,17.82,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2,0.50,11.06,18.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X

array([[ 1.4477984 ,  1.02985697, -1.51052587, ..., -0.3870302 ,
        -0.08748022, -0.02823612],
       [ 0.14340125,  0.75274812, -1.01823211, ...,  2.58377767,
        -0.08748022, -0.02823612],
       [ 0.14340125, -1.22165246,  1.03311991, ..., -0.3870302 ,
        -0.08748022, -0.02823612],
       ...,
       [-1.1609959 ,  1.6879905 ,  0.05237243, ..., -0.3870302 ,
        -0.08748022, -0.02823612],
       [-1.1609959 ,  0.16389181,  1.33417944, ..., -0.3870302 ,
        -0.08748022, -0.02823612],
       [-1.1609959 ,  1.23768861,  0.76431677, ..., -0.3870302 ,
        -0.08748022, -0.02823612]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y1, test_size=0.15, shuffle=True)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)



Train set: (16004, 59) (16004, 4)
Test set: (2825, 59) (2825, 4)


In [ ]:
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ANN(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=59, out_features=100,bias=True)
    self.layer_2= nn.Linear(in_features=100, out_features=80,bias=True)
    self.layer_3 = nn.Linear(in_features=80, out_features=64,bias=True)
    self.output = nn.Linear(in_features=64, out_features=3)
    
    self.logprob = nn.LogSoftmax(dim=1)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=0.2)
    self.dropout = nn.Dropout(p=0.2)
    self.batchnorm1 = nn.BatchNorm1d(100)
    self.batchnorm2 = nn.BatchNorm1d(80)
    self.batchnorm3 = nn.BatchNorm1d(64)
  def forward(self, x):
    x = self.layer_1(x)
    x = self.batchnorm1(x)
    x = self.relu(x)
        
    x = self.layer_2(x)
    x = self.batchnorm2(x)
    x = self.relu(x)
    x = self.dropout(x)
        
    x = self.layer_3(x)
    x = self.batchnorm3(x)
    x = self.relu(x)
    x = self.dropout(x)
    
    #x = self.output(x).view(-1)
    x = self.output(x)
    x = self.logprob(x)
    #x = x.to(torch.int64)
    return x
    

In [ ]:
class ANNsoft(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=59, out_features=100,bias=True)
    self.layer_2= nn.Linear(in_features=100, out_features=80,bias=True)
    self.layer_3 = nn.Linear(in_features=80, out_features=64,bias=True)
    self.output = nn.Linear(in_features=64, out_features=4)
    
    
    
    self.logprob= nn.LogSoftmax(dim=1)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=0.1)
    self.dropout = nn.Dropout(p=0.1)
    #self.batchnorm1 = nn.BatchNorm1d(100)
    #self.batchnorm2 = nn.BatchNorm1d(80)
    #self.batchnorm3 = nn.BatchNorm1d(64)
  def forward(self, x):
    x = self.layer_1(x)
    #x = self.batchnorm1(x)
    x = self.relu(x)
        
    x = self.layer_2(x)
    #x = self.batchnorm2(x)
    x = self.relu(x)
    x = self.dropout(x)
        
    x = self.layer_3(x)
    #x = self.batchnorm3(x)
    x = self.relu(x)
    x = self.dropout(x)
    #x = self.output(x).view(-1)
    x = self.output(x)
    x = self.logprob(x)
    return x
    

In [ ]:
#let's define the ANN with softmax as last layer
model2=ANNsoft()

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train.to_numpy())
y_test = torch.LongTensor(y_test.to_numpy())

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#loss_function = nn.CrossEntropyLoss()
loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)
#optimizer=torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
from sklearn.metrics import f1_score
#loss_function=nn.MSELoss()

y_train.size()

torch.Size([16004, 4])

In [ ]:
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

In [ ]:
from statistics import mean 
def Average(lst): 
    return mean(lst) 



In [ ]:
import torch.utils.data as data_utils
#my_train = data_utils.TensorDataset(X_train, y_train)
train_loader = data_utils.DataLoader(my_train,batch_size=1,shuffle=True)


In [ ]:
model2=torch.load('/content/model2loader(1).pt')

In [ ]:
epochs = 3

loss_arr = []
f1_arr = []
for i in range(epochs):
  for batch in train_loader:
    X_train, y_train = batch
#for i, (X_train, y_train) in enumerate(train_loader):
   
   #X_train, X_test, y_train, y_test = train_test_split( X, y1, test_size=0.2, shuffle=True)

   #X_train = torch.FloatTensor(X_train)
   #X_test = torch.FloatTensor(X_test)
   #y_train = torch.LongTensor(y_train.to_numpy())
   
   #y_test = torch.LongTensor(y_test.to_numpy())
   
    y_hat = model2.forward(X_train)
   
    y_train=torch.argmax(y_train,dim=1)
   
    loss = loss_function(y_hat,y_train)
   #print(y_hat,y_train,loss)

    y_hat=torch.argmax(y_hat,dim=1)
   #y_hat[y_hat == 3]=4
    yt=y_train.data.numpy()
    yp=y_hat.data.numpy().astype('int64')
   #yp=y_hat.data.numpy()
   
    f1 = f1_score(yt,yp,average='weighted')
   

   #f1=np.array(f1)
   #loss=torch.from_numpy(f1).to(torch.float32)
   #loss=torch.tensor(loss,requires_grad=True)
    loss_arr.append(loss) 
    f1_arr.append(f1)

    if i % 10 == 0:
       print(f'Epoch: {i} F1 score: {f1} Loss: {loss}')
       #print(yp, yt)
       print(Average(f1_arr))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Output streaming troncato alle ultime 5000 righe.
0.8994446501295816
Epoch: 0 F1 score: 1.0 Loss: 0.09564653038978577
0.8994520953650229
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.899459539498038
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.8994669825288718
Epoch: 0 F1 score: 1.0 Loss: 0.0923217162489891
0.8994744244577689
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.8994818652849741
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.899489305010732
Epoch: 0 F1 score: 0.0 Loss: 1.1425706148147583
0.8994227353463587
Epoch: 0 F1 score: 0.0 Loss: 0.9410808682441711
0.8993561755346703
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.8993636229095753
Epoch: 0 F1 score: 1.0 Loss: 0.03532780334353447
0.89937106918239
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.899378514353359
Epoch: 0 F1 score: 0.0 Loss: 0.7629668712615967
0.8993119775098025
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.8993194259505845
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.8993268732894445
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.8993343195266272
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.899341764662377
Epoch: 0 F1 s

In [ ]:
my_test = data_utils.TensorDataset(X_test, y_test)
test_loader = data_utils.DataLoader(my_test,batch_size=30,shuffle=True)
model2.eval()

ANNsoft(
  (layer_1): Linear(in_features=59, out_features=100, bias=True)
  (layer_2): Linear(in_features=100, out_features=80, bias=True)
  (layer_3): Linear(in_features=80, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=4, bias=True)
  (logprob): LogSoftmax(dim=1)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
test_loader = data_utils.DataLoader(my_test,batch_size=243,shuffle=True)

In [ ]:
y_pred = model2(X_test)
y_pred = torch.argmax(y_pred,dim=1)
y_pred[y_pred == 3]=4
y_pred 
y_test=torch.argmax(y_test,dim=1)
y_test[y_test == 3]=4
yt_test=y_test.data.numpy()
yp_test=y_pred.data.numpy().astype('int64')
after_train = f1_score(yt_test,yp_test,average='weighted')
print('Test loss after Training' , after_train.item())

IndexError: ignored

In [ ]:
average=[]
for batch in test_loader:
  X_test,y_test=batch

  y_pred = model2(X_test)
  y_pred = torch.argmax(y_pred,dim=1)
  y_pred[y_pred == 3]=4
  y_pred 
  y_test=torch.argmax(y_test,dim=1)
  y_test[y_test == 3]=4
  yt_test=y_test.data.numpy()
  yp_test=y_pred.data.numpy().astype('int64')
  after_train = f1_score(yt_test,yp_test,average='weighted')
  average.append(after_train)
print('Test loss after Training' , Average(average))

Test loss after Training 0.8982588991967333


In [ ]:
arr = y_pred.data.cpu().numpy()
arr2 = y_test.data.cpu().numpy()
np.savetxt('outputsoftmax.csv',  np.column_stack((arr2, arr)), delimiter=',')
    

In [ ]:
PATH='/content/model2loader.pt'

In [ ]:
torch.save(model2,PATH)

In [ ]:
test_set.shape

(8072, 9)

In [ ]:
test_set=pd.read_csv('Dataset/test.csv')
X=test_set
Feature_test = pd.get_dummies(X['color_type'])
X.drop(columns=['issue_date','listing_date','pet_id'], inplace=True)
X.drop(columns=['color_type'], inplace=True)
X=pd.concat([X,Feature_test],axis=1)
columns=X.columns
imputer = KNNImputer(n_neighbors=3, weights='uniform', metric='nan_euclidean')
X=imputer.fit_transform(X)

X=pd.DataFrame(data=X, columns=columns)
X['condition']=X['condition'].astype('int')
X.head()

,condition,length(m),height(cm),X1,X2,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,Gray Tabby,Green,Lilac Point,Liver,Liver Tick,Lynx Point,Orange,Orange Tabby,Pink,Red,Red Merle,Red Tick,Sable,Seal Point,Silver,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,0,0.87,42.73,0.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.06,6.71,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.24,41.21,0.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.29,8.46,7.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.71,30.92,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X

array([[-1.18153612,  1.25255298,  1.18283777, ..., -0.38308459,
        -0.09009937, -0.03149704],
       [ 0.13016235, -1.54443889, -1.6057133 , ..., -0.38308459,
        -0.09009937, -0.03149704],
       [ 0.13016235, -0.92288514,  1.06516432, ..., -0.38308459,
        -0.09009937, -0.03149704],
       ...,
       [-1.18153612,  1.63239138,  0.75394902, ..., -0.38308459,
        -0.09009937, -0.03149704],
       [-1.18153612,  0.97630687, -0.28033867, ..., -0.38308459,
        -0.09009937, -0.03149704],
       [-1.18153612,  0.45834541, -0.22769528, ..., -0.38308459,
        -0.09009937, -0.03149704]])

In [ ]:
X_test_1 = torch.FloatTensor(X)
model2.eval()
y_pred_test = model2(X_test_1)

In [ ]:
y_pred_test=torch.argmax(y_pred_test,dim=1)




In [ ]:
y_pred_test[y_pred_test==3]=4
y_pred_test

tensor([2, 1, 2,  ..., 2, 2, 2])

In [ ]:
test_set=pd.read_csv('Dataset/test.csv')
test_set['pet_category']=y_pred_test
test_set

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,pet_category
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7,2
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7,2
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1,2
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7,2
...,...,...,...,...,...,...,...,...,...,...
8067,ANSL_66809,2016-02-10 00:00:00,2017-03-10 14:56:00,2.0,Brown,0.82,36.08,13,9,2
8068,ANSL_59041,2015-12-07 00:00:00,2018-02-12 00:00:00,0.0,Tan,0.49,27.54,13,9,2
8069,ANSL_60034,2015-12-08 00:00:00,2017-01-04 17:19:00,0.0,Black,0.98,37.19,0,7,2
8070,ANSL_58066,2016-06-28 00:00:00,2017-07-20 18:19:00,NaN,Black,0.79,23.83,0,2,2


In [ ]:
#final_df=test_set[['pet_id','pet_category']].to_csv('final_output.csv',index=False)
final_df=test_set[['pet_id','pet_category']]
final_df.head()

,pet_id,pet_category
0,ANSL_75005,2
1,ANSL_76663,1
2,ANSL_58259,2
3,ANSL_67171,2
4,ANSL_72871,2


In [ ]:
train_set.shape

(18834, 63)

In [ ]:
X

array([[-1.18153612,  1.25255298,  1.18283777, ..., -0.38308459,
        -0.09009937, -0.03149704],
       [ 0.13016235, -1.54443889, -1.6057133 , ..., -0.38308459,
        -0.09009937, -0.03149704],
       [ 0.13016235, -0.92288514,  1.06516432, ..., -0.38308459,
        -0.09009937, -0.03149704],
       ...,
       [-1.18153612,  1.63239138,  0.75394902, ..., -0.38308459,
        -0.09009937, -0.03149704],
       [-1.18153612,  0.97630687, -0.28033867, ..., -0.38308459,
        -0.09009937, -0.03149704],
       [-1.18153612,  0.45834541, -0.22769528, ..., -0.38308459,
        -0.09009937, -0.03149704]])

In [ ]:
train_set=pd.read_csv('Dataset/train.csv')
columns=train_set.columns
X=train_set[columns]

X=X.drop(X[(X['Black Tiger'] == 1) | (X['Brown Tiger'] == 1)].index)
#y2=pd.get_dummies(X['breed_category'])
#X.drop(['breed_category','pet_category','Black Tiger','Brown Tiger'],axis=1,inplace=True)
X.head()


AttributeError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y2_train, y2_test = train_test_split( X, y2, test_size=0.15)
print ('Train set:', X_train.shape,  y2_train.shape)
print ('Test set:', X_test.shape,  y2_test.shape)

Train set: (16004, 59) (16004, 3)
Test set: (2825, 59) (2825, 3)


In [ ]:
model=ANN()

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y2_train = torch.LongTensor(y2_train.to_numpy())
y2_test = torch.LongTensor(y2_test.to_numpy())

In [ ]:
#loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#optimizer=torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)<
from sklearn.metrics import f1_score
#loss_function=nn.MSELoss()
loss_function=nn.NLLLoss()

In [ ]:
y2_train=torch.argmax(y2_train,dim=1)
X_train

tensor([[ 0.1434, -1.5334,  0.7520,  ..., -0.3870, -0.0875, -0.0282],
        [ 0.1434,  1.4109, -0.0383,  ..., -0.3870, -0.0875, -0.0282],
        [ 0.1434,  0.0253,  0.7551,  ..., -0.3870, -0.0875, -0.0282],
        ...,
        [-1.1610, -1.1177, -1.7064,  ..., -0.3870, -0.0875, -0.0282],
        [ 0.1434, -1.7066, -0.1842,  ..., -0.3870, -0.0875, -0.0282],
        [ 1.4478,  0.9259, -1.1949,  ..., -0.3870, -0.0875, -0.0282]])

In [ ]:
import torch.utils.data as data_utils
my_train = data_utils.TensorDataset(X_train, y2_train)
train_loader = data_utils.DataLoader(my_train,batch_size=10,shuffle=True)

In [118]:
train_loader = data_utils.DataLoader(my_train,batch_size=1,shuffle=True)

In [119]:
epochs = 1

loss_arr = []
f1_arr = []
for i in range(epochs):
   #X_train, X_test, y2_train, y2_test = train_test_split( X, y2, test_size=0.2)
   #X_train = torch.FloatTensor(X_train)
   #X_test = torch.FloatTensor(X_test)
   #y2_train = torch.LongTensor(y2_train.to_numpy())
   #y2_test = torch.LongTensor(y2_test.to_numpy())
   for batch in train_loader:
     X_train,y2_train=batch
     y_hat = model.forward(X_train)
     #print(y_hat.grad_fn)
     y2_train=torch.argmax(y2_train,dim=1)
     loss = loss_function(y_hat,y2_train)
     y_hat=torch.argmax(y_hat,dim=1)
     yt=y2_train.data.numpy()
     yp=y_hat.data.numpy().astype('int64')
     f1 = f1_score(yt,yp,average='weighted')
     loss_arr.append(loss)
     f1_arr.append(f1)
     if i % 1 == 0:
          print(f'Epoch: {i} F1 score: {f1} Loss: {loss}')
          #print(yp, yt)
          print(Average(f1_arr))
     optimizer.zero_grad()
     loss.backward()
     optimizer.step()
     
      
      
      #loss = loss_function(y_hat,y2_train)
      #print(y_hat,y2_train,loss)
      
      #yp=y_hat.data.numpy()
      
      


      
      #f1=np.array(f1)
      #loss=torch.from_numpy(f1).to(torch.float32)
      #loss=torch.tensor(loss,requires_grad=True)
       
      

    
          
          
      

Output streaming troncato alle ultime 5000 righe.
0.8553128470936691
Epoch: 0 F1 score: 1.0 Loss: 9.536738616588991e-07
0.855323559899304
Epoch: 0 F1 score: 1.0 Loss: 0.00013433984713628888
0.8553342711186792
Epoch: 0 F1 score: 1.0 Loss: 0.00038044367101974785
0.8553449807521468
Epoch: 0 F1 score: 1.0 Loss: 0.7447977662086487
0.8553556888000592
Epoch: 0 F1 score: 1.0 Loss: 8.344646857949556e-07
0.8553663952627684
Epoch: 0 F1 score: 1.0 Loss: 8.702239938429557e-06
0.8553771001406262
Epoch: 0 F1 score: 1.0 Loss: 0.0001255195093108341
0.8553878034339846
Epoch: 0 F1 score: 1.0 Loss: 0.0
0.8553985051431955
Epoch: 0 F1 score: 1.0 Loss: 0.23433321714401245
0.8554092052686103
Epoch: 0 F1 score: 1.0 Loss: 0.00023862851958256215
0.8554199038105809
Epoch: 0 F1 score: 1.0 Loss: 0.7307299375534058
0.8554306007694584
Epoch: 0 F1 score: 0.0 Loss: 0.8862582445144653
0.85536731523267
Epoch: 0 F1 score: 0.0 Loss: 0.76640385389328
0.8553040390590324
Epoch: 0 F1 score: 1.0 Loss: 0.10925910621881485
0.8553

In [120]:
model.eval()
y_pred = model(X_test)
y_pred = torch.argmax(y_pred,dim=1)
yt_test = torch.argmax(y2_test, dim=1)
yt_test=yt_test.data.numpy()
yp_test=y_pred.data.numpy().astype('int64')
after_train2 = f1_score(yt_test,yp_test,average='weighted')
print('Test loss after Training' , after_train2.item())

Test loss after Training 0.8569007767710659


In [122]:
#Final accuracy score for model and model2
final_score=[]
final_score.append(after_train)
final_score.append(after_train2)
final_scores=Average(final_score)
final_scores

0.928450388385533

In [123]:
test_set=pd.read_csv('Dataset/test.csv')
X=test_set
X.drop(columns=['issue_date','listing_date','pet_id'], inplace=True)
Feature_test = pd.get_dummies(X['color_type'])
X.drop(columns=['color_type'], inplace=True)
X=pd.concat([X,Feature_test],axis=1)
columns=X.columns

In [124]:
X=imputer.fit_transform(X)
X=pd.DataFrame(data=X,columns=columns)
X.head()

,condition,length(m),height(cm),X1,X2,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,Gray Tabby,Green,Lilac Point,Liver,Liver Tick,Lynx Point,Orange,Orange Tabby,Pink,Red,Red Merle,Red Tick,Sable,Seal Point,Silver,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,0.0,0.87,42.73,0.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.06,6.71,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.24,41.21,0.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.29,8.46,7.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.71,30.92,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [125]:
X= preprocessing.StandardScaler().fit(X).transform(X)


In [126]:
test_set.shape

(8072, 5)

In [127]:
X_test_2 = torch.FloatTensor(X)
model.eval()
y_pred_test = model(X_test_2)

In [130]:
torch.save(model, '/content/modelloader.pt')

In [ ]:
y_pred_test

tensor([[-4.4572e+00, -2.2799e-02, -4.5148e+00],
        [-3.6888e-04, -8.1475e+00, -9.4422e+00],
        [-9.3575e-05, -9.9405e+00, -9.9986e+00],
        ...,
        [-4.4338e+00, -2.2500e-02, -4.5679e+00],
        [-7.1894e-01, -1.0377e+00, -1.8422e+00],
        [-4.9230e+00, -1.4264e-02, -4.9784e+00]], grad_fn=<LogSoftmaxBackward>)

In [128]:
y_pred_test=torch.argmax(y_pred_test,dim=1)


In [129]:
final_df['breed_category']=y_pred_test

final_df=final_df[['pet_id','breed_category','pet_category']]
final_df.to_csv('/content/final_output.csv', index=False)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
